In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import pickle
import nltk
import time
import warnings
import re
from functions import (cosine_similarity, get_dict, process_tweet)
from nltk.corpus import stopwords, twitter_samples
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('twitter_samples')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /Users/vuhan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/vuhan/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


<a name="1"></a>

# 1. The Word Embeddings Data for English and French Words

Write a program that translates English to French.

In [2]:
en_embeddings_subset = pickle.load(open("./data/en_embeddings.p", "rb"))
fr_embeddings_subset = pickle.load(open("./data/fr_embeddings.p", "rb"))

#### Look at the data

* en_embeddings_subset: the key is an English word, and the value is a
300 dimensional array, which is the embedding for that word.
```
'the': array([ 0.08007812,  0.10498047,  0.04980469,  0.0534668 , -0.06738281, ....
```

* fr_embeddings_subset: the key is a French word, and the value is a 300
dimensional array, which is the embedding for that word.
```
'la': array([-6.18250e-03, -9.43867e-04, -8.82648e-03,  3.24623e-02,...
```

#### Load two dictionaries mapping the English to French words
* A training dictionary
* and a testing dictionary.

In [3]:
# loading the english to french dictionaries
en_fr_train = get_dict('./data/en-fr.train.txt')
print('The length of the English to French training dictionary is', len(en_fr_train))
en_fr_test = get_dict('./data/en-fr.test.txt')
print('The length of the English to French test dictionary is', len(en_fr_test))

The length of the English to French training dictionary is 5000
The length of the English to French test dictionary is 1500


#### Looking at the English French dictionary

* `en_fr_train` is a dictionary where the key is the English word and the value
is the French translation of that English word.
```
{'the': 'la',
 'and': 'et',
 'was': 'était',
 'for': 'pour',
```

* `en_fr_test` is similar to `en_fr_train`, but is a test set.  We won't look at it
until we get to testing.

<a name="1-1"></a>
### 1.1 Generate Embedding and Transform Matrices

<a name="ex-1"></a>
### Function `get_matrices`

Translating English dictionary to French by using embeddings.

Implement a function `get_matrices`, which takes the loaded data
and returns matrices `X` and `Y`.

Inputs:
- `en_fr` : English to French dictionary
- `en_embeddings` : English to embeddings dictionary
- `fr_embeddings` : French to embeddings dictionary

Returns:
- Matrix `X` and matrix `Y`, where each row in X is the word embedding for an
english word, and the same row in Y is the word embedding for the French
version of that English word.

In [4]:
def get_matrices(en_fr, fr_vecs, eng_vecs):
    """
    Input:
        en_fr: English to French dictionary
        fr_vecs: French words to their corresponding word embeddings.
        eng_vecs: English words to their corresponding word embeddings.
    Output: 
        X: a matrix where the columns are the English embeddings.
        Y: a matrix where the columns correspong to the French embeddings.
        R: the projection matrix that minimizes the F norm ||X R -Y||^2.
    """
    
    lst_X = list()
    lst_Y = list()
    
    eng_set = set(eng_vecs.keys())
    fren_set = set(fr_vecs.keys())
    fren_words = set(en_fr.values())
    
    for en_word, fr_word in en_fr.items():
        if en_word in eng_set and fr_word in fren_set:
            en_vec = eng_vecs[en_word]
            fr_vec = fr_vecs[fr_word]
            
            lst_X.append(en_vec)
            lst_Y.append(fr_vec)
            
    X = np.vstack(lst_X)
    Y = np.vstack(lst_Y)
    
    return X, Y

In [5]:
X_train, y_train = get_matrices(en_fr_train, fr_embeddings_subset, en_embeddings_subset)

In [6]:
print(X_train.shape)
print(y_train.shape)

(4932, 300)
(4932, 300)


<a name="2"></a>

# 2. Translations

Write a program that translates English words to French words using word embeddings and vector space models. 

<a name="2-1"></a>
### 2.1 Translation as Linear Transformation of Embeddings

Given dictionaries of English and French word embeddings then create a transformation matrix `R`
* Given an English word embedding, $\mathbf{e}$, multiply $\mathbf{eR}$ to get a new word embedding $\mathbf{f}$.
    * Both $\mathbf{e}$ and $\mathbf{f}$ are [row vectors](https://en.wikipedia.org/wiki/Row_and_column_vectors).
* We can then compute the nearest neighbors to `f` in the french embeddings and recommend the word that is most similar to the transformed word embedding.

#### Describing translation as the minimization problem

Find a matrix `R` that minimizes the following equation. 

$$\arg \min _{\mathbf{R}}\| \mathbf{X R} - \mathbf{Y}\|_{F}\tag{1} $$

#### Frobenius norm

The Frobenius norm of a matrix $A$ (assuming it is of dimension $m,n$) is defined as the square root of the sum of the absolute squares of its elements:

$$\|\mathbf{A}\|_{F} \equiv \sqrt{\sum_{i=1}^{m} \sum_{j=1}^{n}\left|a_{i j}\right|^{2}}\tag{2}$$

#### Actual loss function
In the real world applications, the Frobenius norm loss:

$$\| \mathbf{XR} - \mathbf{Y}\|_{F}$$

is often replaced by it's squared value divided by $m$:

$$ \frac{1}{m} \|  \mathbf{X R} - \mathbf{Y} \|_{F}^{2}$$

where $m$ is the number of examples (rows in $\mathbf{X}$).

* The same R is found when using this loss function versus the original Frobenius norm.
* The reason for taking the square is that it's easier to compute the gradient of the squared Frobenius.
* The reason for dividing by $m$ is that we're more interested in the average loss per embedding than the  loss for the entire training set.
    * The loss for all training set increases with more words (training examples),
    so taking the average helps us to track the average loss regardless of the size of the training set.

##### Detailed explanation why we use norm squared instead of the norm:

- The norm is always nonnegative (we're summing up absolute values), and so is the square.
- When we take the square of all non-negative (positive or zero) numbers, the order of the data is preserved.
- For example, if 3 > 2, 3^2 > 2^2
- Using the norm or squared norm in gradient descent results in the same location of the minimum.
- Squaring cancels the square root in the Frobenius norm formula. Because of the chain rule, we would have to do more calculations if we had a square root in our expression for summation.
- Dividing the function value by the positive number doesn't change the optimum of the function, for the same reason as described above.
- We're interested in transforming English embedding into the French. Thus, it is more important to measure average loss per embedding than the loss for the entire dictionary (which increases as the number of words in the dictionary increases).

#### Implementing translation mechanism described in this section.


### Function `compute_loss`

#### Step 1: Computing the loss
* The loss function will be squared Frobenius norm of the difference between
matrix and its approximation, divided by the number of training examples $m$.
* Its formula is:
$$ L(X, Y, R)=\frac{1}{m}\sum_{i=1}^{m} \sum_{j=1}^{n}\left( a_{i j} \right)^{2}$$

where $a_{i j}$ is value in $i$th row and $j$th column of the matrix $\mathbf{XR}-\mathbf{Y}$.

In [7]:
def compute_loss(X, Y, R):
    '''
    Inputs: 
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        R: a matrix of dimension (n,n) - transformation matrix from English to French vector space embeddings.
    Outputs:
        L: a matrix of dimension (m,n) - the value of the loss function for given X, Y and R.
    '''
    
    m = X.shape[0]
    
    diff = np.dot(X, R) - Y
    
    diff_squared = np.square(diff)
    
    sum_diff_squared = np.sum(diff_squared)
    
    loss = sum_diff_squared / m
    
    return loss

In [8]:
np.random.seed(42)

m = 10
n = 5
X = np.random.rand(m, n)
Y = np.random.rand(m, n) * .1
R = np.random.rand(n, n)

exp_loss = compute_loss(X, Y, R)

print('Expected loss for experiment with random matrix: {:.2f}'.format(exp_loss))

Expected loss for experiment with random matrix: 6.56



### Function `compute_gradient`

#### Step 2: Computing the gradient of loss with respect to transform matrix R

* Calculate the gradient of the loss with respect to transform matrix `R`.
* The gradient is a matrix that encodes how much a small change in `R`
affect the change in the loss function.
* The gradient gives us the direction in which we should decrease `R`
to minimize the loss.
* $m$ is the number of training examples (number of rows in $X$).
* The formula for the gradient of the loss function $𝐿(𝑋,𝑌,𝑅)$ is:

$$\frac{d}{dR}𝐿(𝑋,𝑌,𝑅)=\frac{d}{dR}\Big(\frac{1}{m}\| X R -Y\|_{F}^{2}\Big) = \frac{2}{m}X^{T} (X R - Y)$$

In [9]:
def compute_gradient(X, Y, R):
    '''
    Inputs: 
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        R: a matrix of dimension (n,n) - transformation matrix from English to French vector space embeddings.
    Outputs:
        g: a scalar value - gradient of the loss function L for given X, Y and R.
    '''
    
    m = X.shape[0]
    
    gradient = np.dot(X.T, (np.dot(X, R) - Y)) * 2 / m
    
    return gradient

In [10]:
np.random.seed(42)

m = 10
n = 5
X = np.random.rand(m, n)
Y = np.random.rand(m, n) * .1
R = np.random.rand(n, n)

exp_gradient = compute_gradient(X, Y, R)

print('First row of gradient matrix: {}'.format(exp_gradient[0]))

First row of gradient matrix: [0.8509425  1.17397439 0.82708681 1.06200256 1.03393491]


### Function `align_embeddings`

#### Step 3: Finding the optimal R with Gradient Descent Algorithm

##### Gradient Descent

[Gradient descent](https://ml-cheatsheet.readthedocs.io/en/latest/gradient_descent.html) is an iterative algorithm which is used in searching for the optimum of the function. 
* Earlier, we've mentioned that the gradient of the loss with respect to the matrix encodes how much a tiny change in some coordinate of that matrix affect the change of loss function.
* Gradient descent uses that information to iteratively change matrix `R` until we reach a point where the loss is minimized. 

Pseudocode:
1. Calculate gradient $g$ of the loss with respect to the matrix $R$.
2. Update $R$ with the formula:
$$R_{\text{new}}= R_{\text{old}}-\alpha g$$

Where $\alpha$ is the learning rate, which is a scalar.

#### Training with a fixed number of iterations

Most of the time we iterate for a fixed number of training steps rather than iterating until the loss falls below a threshold.

##### Explanation for fixed number of iterations

- You cannot rely on training loss getting low -- what you really want is the validation loss to go down, or validation accuracy to go up. And indeed - in some cases people train until validation accuracy reaches a threshold, or -- commonly known as "early stopping" -- until the validation accuracy starts to go down, which is a sign of over-fitting.
- Why not always do "early stopping"? Well, mostly because well-regularized models on larger data-sets never stop improving. Especially in NLP, you can often continue training for months and the model will continue getting slightly and slightly better. This is also the reason why it's hard to just stop at a threshold -- unless there's an external customer setting the threshold, why stop, where do you put the threshold?
- Stopping after a certain number of steps has the advantage that you know how long your training will take - so you can keep some sanity and not train for months. You can then try to get the best performance within this time budget. Another advantage is that you can fix your learning rate schedule -- e.g., lower the learning rate at 10% before finish, and then again more at 1% before finishing. Such learning rate schedules help a lot, but are harder to do if you don't know how long you're training.

In [11]:
def align_embeddings(X, Y, num_iterations=300, learning_rate=0.0005, verbose=True, 
                     compute_loss=compute_loss, compute_gradient=compute_gradient):
    '''
    Inputs:
        X: a matrix of dimension (m,n) where the columns are the English embeddings.
        Y: a matrix of dimension (m,n) where the columns correspong to the French embeddings.
        num_iterations: positive int - describes how many steps will gradient descent algorithm do.
        learning_rate: positive float - describes how big steps will  gradient descent algorithm do.
    Outputs:
        R: a matrix of dimension (n,n) - the projection matrix that minimizes the F norm ||X R -Y||^2
    '''
    
    np.random.seed(42)
    
    R = np.random.rand(X.shape[1], X.shape[1])
    
    for i in range(num_iterations):
        if verbose and i % 25 == 0:
            print('Loss at iteration {} is: {:.4f}'.format(i, compute_loss(X, Y, R)))
        
        gradient = compute_gradient(X, Y, R)
        
        R = R - (learning_rate * gradient)
        
    return R

In [12]:
np.random.seed(42)

m = 10
n = 5
X = np.random.rand(m, n)
Y = np.random.rand(m, n) * .1

exp_R = align_embeddings(X, Y)

Loss at iteration 0 is: 4.9403
Loss at iteration 25 is: 4.6943
Loss at iteration 50 is: 4.4608
Loss at iteration 75 is: 4.2393
Loss at iteration 100 is: 4.0290
Loss at iteration 125 is: 3.8294
Loss at iteration 150 is: 3.6400
Loss at iteration 175 is: 3.4602
Loss at iteration 200 is: 3.2895
Loss at iteration 225 is: 3.1276
Loss at iteration 250 is: 2.9738
Loss at iteration 275 is: 2.8279


#### Calculate Transformation matrix R

Using just the training set, find the transformation matrix $\mathbf{R}$ by calling the function `align_embeddings()`.

In [19]:
R_train = align_embeddings(X_train, y_train, num_iterations=800, learning_rate=0.9)

Loss at iteration 0 is: 966.0546
Loss at iteration 25 is: 80.9353
Loss at iteration 50 is: 20.2760
Loss at iteration 75 is: 7.0565
Loss at iteration 100 is: 3.1239
Loss at iteration 125 is: 1.7019
Loss at iteration 150 is: 1.1137
Loss at iteration 175 is: 0.8461
Loss at iteration 200 is: 0.7157
Loss at iteration 225 is: 0.6488
Loss at iteration 250 is: 0.6130
Loss at iteration 275 is: 0.5932
Loss at iteration 300 is: 0.5819
Loss at iteration 325 is: 0.5753
Loss at iteration 350 is: 0.5713
Loss at iteration 375 is: 0.5689
Loss at iteration 400 is: 0.5674
Loss at iteration 425 is: 0.5664
Loss at iteration 450 is: 0.5658
Loss at iteration 475 is: 0.5654
Loss at iteration 500 is: 0.5652
Loss at iteration 525 is: 0.5650
Loss at iteration 550 is: 0.5649
Loss at iteration 575 is: 0.5648
Loss at iteration 600 is: 0.5648
Loss at iteration 625 is: 0.5648
Loss at iteration 650 is: 0.5647
Loss at iteration 675 is: 0.5647
Loss at iteration 700 is: 0.5647
Loss at iteration 725 is: 0.5647
Loss at ite

### 2.2 Testing the Translation
​
#### k-Nearest Neighbors Algorithm
​
[k-Nearest neighbors algorithm](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) 
* k-NN is a method which takes a vector as input and finds the other vectors in the dataset that are closest to it. 
* The 'k' is the number of "nearest neighbors" to find (e.g. k=2 finds the closest two neighbors).
​
#### Searching for the Translation Embedding
Since we're approximating the translation function from English to French embeddings by a linear transformation matrix $\mathbf{R}$, most of the time we won't get the exact embedding of a French word when we transform embedding $\mathbf{e}$ of some particular English word into the French embedding space. 
* This is where $k$-NN becomes really useful! By using $1$-NN with $\mathbf{eR}$ as input, we can search for an embedding $\mathbf{f}$ (as a row) in the matrix $\mathbf{Y}$ which is the closest to the transformed vector $\mathbf{eR}$

#### Cosine Similarity
Cosine similarity between vectors $u$ and $v$ calculated as the cosine of the angle between them.
The formula is 

$$\cos(u,v)=\frac{u\cdot v}{\left\|u\right\|\left\|v\right\|}$$

* $\cos(u,v)$ = $1$ when $u$ and $v$ lie on the same line and have the same direction.
* $\cos(u,v)$ is $-1$ when they have exactly opposite directions.
* $\cos(u,v)$ is $0$ when the vectors are orthogonal (perpendicular) to each other.

#### Distance and similarity are pretty much opposite things.
* We can obtain distance metric from cosine similarity, but the cosine similarity can't be used directly as the distance metric. 
* When the cosine similarity increases (towards $1$), the "distance" between the two vectors decreases (towards $0$). 
* We can define the cosine distance between $u$ and $v$ as
$$d_{\text{cos}}(u,v)=1-\cos(u,v)$$

### Function `nearest_neighbor`


Inputs:
* Vector `v`,
* A set of possible nearest neighbors `candidates`
* `k` nearest neighbors to find.
* The distance metric should be based on cosine similarity.
* `cosine_similarity` function is already implemented and imported for you. It's arguments are two vectors and it returns the cosine of the angle between them.
* Iterate over rows in `candidates`, and save the result of similarities between current row and vector `v` in a python list. Take care that similarities are in the same order as row vectors of `candidates`.
* Using[numpy argsort]( https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html#numpy.argsort) to sort the indices for the rows of `candidates`.

In [14]:
def nearest_neighbor(v, candidates, k=1, cosine_similarity=cosine_similarity):    
    """
    Input:
      - v, the vector we are going find the nearest neighbor for
      - candidates: a set of vectors where we will find the neighbors
      - k: top k nearest neighbors to find
    Output:
      - k_idx: the indices of the top k closest vectors in sorted form
    """
    
    lst_similarity = []

    for row in candidates:
        cosine_score = cosine_similarity(v, row)
        lst_similarity.append(cosine_score)

    sorted_ids = np.argsort(lst_similarity)
    sorted_ids = sorted_ids[::-1]

    k_idx = sorted_ids[:k]

    return k_idx

In [15]:
v = np.array([1, 0, 1])
candidates = np.array([[1, 0, 5], [-2, 5, 3], [2, 0, 1], [6, -9, 5], [9, 9, 9]])
print(candidates[nearest_neighbor(v, candidates, 3)])

[[2 0 1]
 [1 0 5]
 [9 9 9]]


#### Test Translation and Compute its Accuracy

### Function `test_vocabulary`
Implementing function `test_vocabulary` which takes in English
embedding matrix $X$, French embedding matrix $Y$ and the $R$
matrix and returns the accuracy of translations from $X$ to $Y$ by $R$.

* Iterate over transformed English word embeddings and check if the
closest French word vector belongs to French word that is the actual
translation.
* Obtain an index of the closest French embedding by using
`nearest_neighbor` (with argument `k=1`), and compare it to the index
of the English embedding that have just transformed.
* Keep track of the number of times of the correct translation.
* Calculate accuracy as $$\text{accuracy}=\frac{\#(\text{correct predictions})}{\#(\text{total predictions})}$$

In [16]:
def test_vocabulary(X, Y, R, nearest_neighbor=nearest_neighbor):
    '''
    Input:
        X: a matrix where the columns are the English embeddings.
        Y: a matrix where the columns correspong to the French embeddings.
        R: the transform matrix which translates word embeddings from
        English to French word vector space.
    Output:
        accuracy: for the English to French capitals
    '''
    pred = np.dot(X, R)
    num_correct = 0
    
    for i in range(len(pred)):
        pred_idx = nearest_neighbor(pred[i, :], Y, cosine_similarity=cosine_similarity)
        
        if pred_idx == i:
            num_correct += 1
    accuracy = num_correct / pred.shape[0]
    
    return accuracy

In [17]:
X_test, y_test = get_matrices(en_fr_test, fr_embeddings_subset, en_embeddings_subset)

In [20]:
acc_test = test_vocabulary(X_test, y_test, R_train)

print('Accuracy on test set: {:.3f}'.format(acc_test))

Accuracy on test set: 0.566
